In [0]:
from pyspark.sql.functions import col, isnull, when
from pyspark.sql.types import TimestampType
from datetime import date, timedelta

In [0]:
tiers = ["bronze", "silver", "gold"]
adls_paths = {tier: f"abfss://{tier}@learnbucket.dfs.core.windows.net/" for tier in tiers}

# Accessing paths
bronze_adls = adls_paths["bronze"]
silver_adls = adls_paths["silver"]
gold_adls = adls_paths["gold"]

dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)

In [0]:
start_date = date.today() - timedelta(days=1)
end_date = date.today()

In [0]:
# Catch the data from bronze layer
bronze_output = dbutils.jobs.taskValues.get(taskKey= "Bronze", key="bronze_output")

# Access individual variables
start_date = bronze_output.get("start_date", "")
bronze_adls = bronze_output.get("bronze_adls", "")
silver_adls = bronze_output.get("silver_adls", "")

print(f"Start Date: {start_date}, Bronze ADLS: {bronze_adls}")


In [0]:
# Load the JSON data into a Spark DataFrame
df = spark.read.option("multiline", "true").json(f"{bronze_adls}{start_date}_earthquake_data.json")

In [0]:
df.head()

In [0]:
# Reshape earthquake data
df = (
    df.select(
        "id",
        col('geometry.coordinates').getItem(0).alias('longitude'),
        col('geometry.coordinates').getItem(1).alias('latitude'),
        col('geometry.coordinates').getItem(2).alias('elevation'),
        col('properties.title').alias('title'),
        col('properties.place').alias('place_description'),
        col('properties.sig').alias('sig'),
        col('properties.mag').alias('mag'),
        col('properties.magType').alias('magnitude_type'),
        col('properties.time').alias('time'),
        col('properties.updated').alias('updated')
        )
)

In [0]:
df.head()

In [0]:
# Validate data: Check for missing or null values
df = (
    df
    .withColumn("longitude", when(isnull(col("longitude")), 0).otherwise(col('longitude')))
    .withColumn("latitude", when(isnull(col("latitude")), 0).otherwise(col('latitude')))
    .withColumn("time", when(isnull(col("time")), 0).otherwise(col('time')))
)

In [0]:
# Convert 'time' and 'updated' to timestamp from Unix time
df = (
    df
    .withColumn('time', (col('time') / 1000).cast(TimestampType()))
    .withColumn('updated', (col('updated') / 1000).cast(TimestampType()))
)

In [0]:
df.head()

In [0]:
# Save the transformed DataFrame to the Silver layer
silver_output_path = f"{silver_adls}earthquake_data_silver/"

In [0]:
df.write.mode('append').parquet(silver_output_path)

In [0]:
# Push transformed data to gold layer
dbutils.jobs.taskValues.set(key="silver_output", value = silver_output_path)